## **Importation des données**

In [2]:
from config import *

In [33]:
# Importation des données
df = pd.read_csv(r'C:\Users\HP1\OneDrive\Bureau\PROJETS_DATA\Projet_Credit_Card_Model\Data\Credit-Card-Train.csv')

In [8]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


*Tailles des données*

In [10]:
print(df.shape)

(614, 13)


*Type de données*

In [11]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB
None


 *Description des données*

In [ ]:
#Description des variables numériques 
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [ ]:
#Description des variables catégorielles 
df.describe(include = 'O')

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
count,614,601,611,599,614,582,614,614
unique,614,2,2,4,2,2,3,2
top,LP002990,Male,Yes,0,Graduate,No,Semiurban,Y
freq,1,489,398,345,480,500,233,422


## **Prétraitement des données**

### Vérification et Traitement des valeurs manquantes

* Vérification 

In [26]:
# Calculer la proportion de valeurs manquantes par colonne
missing_proportions= df.isnull().mean()
print("\nLa proportion de données manquantes dans les données d'entrainement sont :\n",missing_proportions)


La proportion de données manquantes dans les données d'entrainement sont :
 Loan_ID              0.000000
Gender               0.021173
Married              0.004886
Dependents           0.024430
Education            0.000000
Self_Employed        0.052117
ApplicantIncome      0.000000
CoapplicantIncome    0.000000
LoanAmount           0.035831
Loan_Amount_Term     0.022801
Credit_History       0.081433
Property_Area        0.000000
Loan_Status          0.000000
dtype: float64


* Traitement

In [34]:
#Séparer le jeu de données en deux, un contenant les variables catégorielles et l'autre les variables numériques
cat_df = df[['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area', 'Loan_Status', 'Credit_History']]
num_df = df[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']]

In [41]:
# Pour les variables catégorielles, remplacer les valeurs manquantes par la valeurs avec le plus grand nombre d'occurences
cat_df=cat_df.apply(lambda x:x.fillna(x.value_counts().index[0]))

# Vérifier s'il reste des valeurs manquantes
print("Reste-t-il des valeurs manquantes ?",cat_df.isnull().sum().any())

Reste-t-il des valeurs manquantes ? False


In [40]:
# Pour les variables numériques, remplacer les valeurs manquantes par la médiane
num_df = num_df.apply(lambda x: x.fillna(x.median()), axis=0)

# Vérifier s'il reste des valeurs manquantes
print("Reste-t-il des valeurs manquantes ?", num_df.isnull().sum().any())

Reste-t-il des valeurs manquantes ? False


In [42]:
# Fusionner les variables catégorielles et numériques
df = pd.concat([num_df, cat_df], axis=1)

*Doublons*

In [43]:
print("Le nombre de doublons dans le jeu de données  est", df.duplicated().sum())

Le nombre de doublons dans le jeu de données  est 0


### Vérification et Traitement des valeurs aberrantes

* Vérification

In [44]:
# Détection des valeurs aberrantes par la méthode de l'IQR
def detect_outliers_iqr(df):
    outlier_percentages = {}

    for column in df.columns:
        Q1 = df[column].quantile(0.25)  # Premier quartile
        Q3 = df[column].quantile(0.75)  # Troisième quartile
        IQR = Q3 - Q1  # Étendue interquartile

        lower_bound = Q1 - 1.5 * IQR  # Seuil inférieur
        upper_bound = Q3 + 1.5 * IQR  # Seuil supérieur

        # Détection des valeurs aberrantes
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        outlier_percentage = (len(outliers) / len(df)) * 100  # Calcul du pourcentage

        outlier_percentages[column] = round(outlier_percentage, 2)  # Arrondi à 2 décimales

    return outlier_percentages

# Appliquer la détection sur num_df
outlier_percentages = detect_outliers_iqr(num_df)

# Affichage des résultats
print("Pourcentage de valeurs aberrantes par colonne dans num_df :")
for col, pct in outlier_percentages.items():
    print(f"{col}: {pct}%")


Pourcentage de valeurs aberrantes par colonne dans num_df :
ApplicantIncome: 8.14%
CoapplicantIncome: 2.93%
LoanAmount: 6.68%
Loan_Amount_Term: 14.33%


* Traitement

Malgré le fait que notre jeux de données contiennent des valeurs aberrantes, nous prenons la décision de les conserver, car elles peuvent être pertinentes pour l'analyse de ces variables , et pourraient refléter des comportements extrêmes ou des événements rares qui sont importants pour la modélisation.

### Sauvegarder le jeu de données nettoyées

In [46]:
#Supprimer Loan_ID
df.drop('Loan_ID', axis=1, inplace=True)

In [48]:
#Sauvegarder les données nettoyées
df.to_csv(r'C:\Users\HP1\OneDrive\Bureau\PROJETS_DATA\Projet_Credit_Card_Model\Data\Credit-Card-Train-Clean.csv', index=False)